In [40]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

import plotly.graph_objects as go

In [41]:
df = pd.read_csv('df_sin_outliers.csv')

In [42]:
# Seleccionamos las columnas que incluiremos en el radar (numericas), ademas añadimos el "identificador" para buscar y "ubicacion" como etiqueta
columnas_importantes = ['identificador', 'superficie', 'superficie_construida', 'superficie_util', 'planta', 'baños', 'precio_m2', 'precio', 'habitaciones', 'ubicacion']
df_importante = df[columnas_importantes].copy()

In [43]:
# Escalamos el precio por separado porque es el valor notablemente mas extremo que desfigura el grafico
scaler_price = RobustScaler()
df_importante['precio_escalado'] = scaler_price.fit_transform(df_importante[['precio']])

# Normalizamos el resto
scaler_others = MinMaxScaler()
df_others_scaled = scaler_others.fit_transform(df_importante.drop(columns=['identificador', 'precio', 'precio_escalado', 'ubicacion']))

# Los combinamos en un unico df
df_scaled = pd.DataFrame(df_others_scaled, columns=columnas_importantes[1:-2])
df_scaled['precio'] = df_importante['precio_escalado']
df_scaled['ubicacion'] = df_importante['ubicacion'].values
df_scaled['identificador'] = df_importante['identificador'].values

In [44]:
df_scaled['identificador'].unique()

array(['33340257518_100500', '50886757575_100500', '51683300341_529013',
       '47517244975_513000', '50856390998_519306', '35018048379_526695',
       '5902921560_109300', '47502779453_528856', '50883275252_519306',
       '50872681561_519306', '50920427457_519306', '50854380584_519306',
       '50901028147_519306', '40925147915_516977', '42509188465_516977',
       '45835622809_995624', '49172728381_512084', '44234791411_100500',
       '41667118296_100500', '35080998400_100500', '21710467295_100500',
       '24207042538_100500', '30022979549_100500', '25856398462_100500',
       '27577902426_100500', '43407410448_100500', '16685628598_100500',
       '22540699556_100500', '37503592408_100500', '37540578316_100500',
       '25073193570_100500', '50864232106_519306', '41686407946_100500',
       '23420993208_100500', '50924911519_100500', '21755871564_100500',
       '21719196290_100500', '485842523_501539', '35028116537_501539',
       '86729224712_501539', '10834785389_501539', '41

In [45]:
# Introducimos manualmente los identificadores a comparar
id1 = '51683300341_529013'
id2 = '51722188486_518438'

In [46]:
# Filtramos
piso1 = df_scaled[df_scaled['identificador'] == id1].drop(columns=['identificador'])
piso2 = df_scaled[df_scaled['identificador'] == id2].drop(columns=['identificador'])

In [47]:
# Creamos el gráfico de radar
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=piso1.iloc[0],
    theta=piso1.columns,
    fill='toself',
    name=f'{id1} - {piso1["ubicacion"].values[0]}'
))

fig.add_trace(go.Scatterpolar(
    r=piso2.iloc[0],
    theta=piso2.columns,
    fill='toself',
    name=f'{id2} - {piso2["ubicacion"].values[0]}'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[-2, 2]
        )
    ),
    showlegend=True
)

fig.show()

Aquí tenemos el grafico comparativo en forma de radar entre las principales caracteristicas de los inmuebles seleccionados.
Ahora añadiremos un df exclusivo de los dos comparados para ver los valores reales de cada categoria esclada.

In [48]:
id1 = '51683300341_529013'
id2 = '51722188486_518438'
piso1_original = df[df['identificador'] == id1][columnas_importantes]
piso2_original = df[df['identificador'] == id2][columnas_importantes]

df_comparativo = pd.concat([piso1_original, piso2_original], ignore_index=True)

In [49]:
df_comparativo

,identificador,superficie,superficie_construida,superficie_util,planta,baños,precio_m2,precio,habitaciones,ubicacion
0,51683300341_529013,79.0,79.0,64.0,2.0,1.0,8.0,650.0,2.0,roquetas_de_mar
1,51722188486_518438,15.0,15.0,14.0,0.0,1.0,30.0,450.0,1.0,cartagena
